In [123]:
import pandas as pd
import requests

def get_headers():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/90.0.4430.212 Safari/537.36'
    }
    return headers

class get_11v11:
    def __init__(self, date, venue = None):
        self.date = date
        self.venue = venue
        
        self.url = self.get_url()

        r = requests.get(self.url, headers = get_headers())

        self.tables = pd.read_html(r.content, flavor="bs4")
        
        self.table = self.get_table()
        self.pos = self.get_pos()

    def get_url(self):
        year = self.date[:4]
        month = pd.to_datetime(self.date).month_name().lower()
        day = self.date[8:]

        url = f"https://www.11v11.com/league-tables/league-two/{day}-{month}-{year}/"
        if self.venue and self.venue[0].upper() in ["H", "A"]:
            if self.venue[0].upper() == "H":
                self.venue = "home"
            else:
                self.venue = "away"
            url += f"{self.venue}"
        return url
    
    def get_table(self):
        table = self.tables[0]
        table.Pos = table.Pos.index + 1
        return table
    
    def get_pos(self):
        try:
            pos = self.table.query("Team.str.contains('Tranmere')").Pos.values[0]
        except:
            pos = "No table containing Tranmere Rovers found"
        return pos

In [124]:
class bbc_api:
    def __init__(self, date):
        self.date = date

        self.match_url = self.get_match_url()
        self.match_list = self.get_match_list()
        self.match_key = self.get_match_key()
        self.event_key = self.get_event_key()
        self.tournament_data = self.get_tournament_data()
        self.match_data = self.get_match_data()

        self.lineup_url = self.get_lineup_url()
        self.lineup_data = self.get_lineup_data()
        self.attendance = self.get_attendance()
        self.referee = self.get_referee()

    def get_match_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-football-scores-match-list-data/endDate/{self.date}/startDate/{self.date}/team/tranmere-rovers/todayDate/{self.date}/version/2.4.6/withPlayerActions/true?timeout=5"
        return url
    
    def get_match_list(self):
        r = requests.get(self.match_url, headers = get_headers())
        match_list = r.json()

        if not match_list['matchData']:
            return f"No matches found for {self.date}"
        else:
            return match_list
    
    def get_match_key(self):
        if self.match_list == f"No matches found for {self.date}":
            return f"No matches found for {self.date}"
        else:
            match_key = next(iter(self.match_list["matchData"][0]["tournamentDatesWithEvents"]))
            return match_key
    
    def get_event_key(self):
        if self.match_list == f"No matches found for {self.date}":
            return f"No matches found for {self.date}"
        else:
            event_key = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]["eventKey"]
            return event_key

    def get_tournament_data(self):
        if self.match_list == f"No matches found for {self.date}":
            return f"No matches found for {self.date}"
        else:
            tournament_data = self.match_list["matchData"][0]["tournamentMeta"]
            return tournament_data    
    
    def get_match_data(self):
        if self.match_list == f"No matches found for {self.date}":
            return f"No matches found for {self.date}"
        else:
            match_data = self.match_list["matchData"][0]["tournamentDatesWithEvents"][self.match_key][0]["events"][0]
            return match_data
        
    def get_lineup_url(self):
        url = f"https://push.api.bbci.co.uk/data/bbc-morph-sport-football-team-lineups-data/event/{self.event_key}/version/1.0.8"
        return url
    
    def get_lineup_data(self):
        r = requests.get(self.lineup_url, headers = get_headers())
        lineup_data = r.json()
        return lineup_data
    
    def get_attendance(self):
        if "attendance" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["attendance"].replace(",", "")
            return attendance
        else:
            print(f"No attendance data for {self.date}")
            return None
    def get_referee(self):
        if "referee" in self.lineup_data["meta"].keys():
            attendance = self.lineup_data["meta"]["referee"].replace(",", "")
            return attendance
        else:
            print(f"No referee for {self.date}")
            return None


In [126]:
date = "2023-08-12"

match_data = bbc_api(date).match_data

teams = ["homeTeam", "awayTeam"]

player_actions = pd.DataFrame()
for team in teams:
    team_name = match_data[team]["name"]["full"]
    try:
        players = match_data[team]["playerActions"]
        for player in players:
            player_name = player["name"]["full"]
            
            actions = player["actions"]
            for action in actions:
                action_df = pd.json_normalize(action)
                action_df["game_date"] = pd.to_datetime(date)
                action_df["team"] = team_name
                action_df["player_name"] = player
                player_actions = pd.concat([player_actions, action_df], axis = 0)
    except KeyError:
        pass
player_actions

,type,timeElapsed,addedTime,penalty,ownGoal,displayTime,game_date,team,player_name
0,goal,7,0,False,False,7',2023-08-12,Milton Keynes Dons,NaN


In [127]:
match_data["venue"]["name"]["full"]

'Stadium MK'

In [128]:
match_data["startTimeInUKHHMM"]

'15:00'